## 🌐 Building an API with Flask

We use Flask to deliver the recommendation model via the `/recommend` endpoint.


In [2]:

import requests
import time

num_requests = 20

test_payload = {
    "co2": 800,
    "humidity": 60
}

# نستخدم localhost بدلاً من ngrok
url = "http://127.0.0.1:5000/recommend"

response_times = []

for i in range(num_requests):
    start_time = time.time()
    response = requests.post(url, json=test_payload)
    end_time = time.time()
    
    duration = end_time - start_time
    response_times.append(duration)
    print(f"Request {i+1}: {duration:.4f} seconds - Status Code: {response.status_code}")

avg_time = sum(response_times) / len(response_times)
print(f"Average Response Time over {num_requests} requests: {avg_time:.4f} seconds")


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /recommend (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000028C39FEA210>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

##  Performance Test

Push 20 requests to the server and measure the latency of each request, then calculate the inflation.


In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(range(1, len(response_times) + 1), response_times, marker='o')
plt.title('Response Time per Request')
plt.xlabel('Request Number')
plt.ylabel('Response Time (seconds)')
plt.grid(True)
plt.show()


## 📊 Response Time Graph

This graph displays the response time for each request to assess stability and performance.


In [ ]:
!pip install pyngrok scikit-learn pandas joblib flask


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2vdEEy402NGHkUfC0LQHmsAzDb6_6nsks9LCY8nfZWhtedQkA")


In [ ]:
import pandas as pd
import numpy as np
import joblib
from flask import Flask, request, jsonify
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, top_k_accuracy_score


In [ ]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(next(iter(uploaded)))
df.head()


In [ ]:
plant_counts = df['common_name'].value_counts()
filtered_df = df[df['common_name'].isin(plant_counts[plant_counts >= 5].index)]
print(f"Original plants: {len(plant_counts)}, After filtering: {filtered_df['common_name'].nunique()}")


In [ ]:
X = filtered_df[['estimated_humidity', 'estimated_co2']]
y = filtered_df['common_name']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

joblib.dump(model, 'plant_recommender_model_humidity_co2.pkl')
joblib.dump(le, 'label_encoder_humidity_co2.pkl')


In [ ]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
top3 = top_k_accuracy_score(y_test, model.predict_proba(X_test), k=3, labels=model.classes_)

print(f"Accuracy: {acc:.2f}")
print("Top-3 Accuracy:", top3)
print("\nClassification Report:")
print(classification_report(
    y_test,
    y_pred,
    labels=np.unique(y_test),
    target_names=le.inverse_transform(np.unique(y_test)).astype(str)
)
)


In [ ]:
app = Flask(__name__)

model = joblib.load('plant_recommender_model_humidity_co2.pkl')
le = joblib.load('label_encoder_humidity_co2.pkl')

@app.route('/recommend', methods=['POST'])
def recommend_plant():
    try:
        data = request.get_json()
        humidity = data['humidity']
        co2 = data['co2']
        probs = model.predict_proba([[humidity, co2]])[0]
        top3_indices = np.argsort(probs)[-3:][::-1]
        top3_plants = le.inverse_transform(top3_indices).tolist()
        return jsonify({"top_3_recommendations": top3_plants})
    except Exception as e:
        return jsonify({"error": str(e)}), 400


public_url = ngrok.connect(5000)
print("🔗 Public URL:", public_url)

app.run()


In [1]:

import requests
import time

num_requests = 20

test_payload = {
    "co2": 800,
    "humidity": 60
}

# نستخدم localhost بدلاً من ngrok
url = "http://127.0.0.1:5000/recommend"  

response_times = []

for i in range(num_requests):
    start_time = time.time()
    response = requests.post(url, json=test_payload)
    end_time = time.time()
    
    duration = end_time - start_time
    response_times.append(duration)
    print(f"Request {i+1}: {duration:.4f} seconds - Status Code: {response.status_code}")

# حساب متوسط زمن الاستجابة
avg_time = sum(response_times) / len(response_times)
print(f"Average Response Time over {num_requests} requests: {avg_time:.4f} seconds")


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /recommend (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001B503E7DBD0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(range(1, len(response_times) + 1), response_times, marker='o')
plt.title('Response Time per Request')
plt.xlabel('Request Number')
plt.ylabel('Response Time (seconds)')
plt.grid(True)
plt.show()
